In [19]:
import music21 as mu
import pandas as pd
import numpy as np
import re

def flength(time):
    threasholds = [0.099, 0.18, 0.30, 0.47, 0.60, 0.81, 1.00, 1.15, 1.30, 1.60, 1.90]
    lengths     = [0.125, 0.25, 0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 2.00, 2.25, 2.50]
    for i in range(len(threasholds)):
        if time < threasholds[i]:
            return lengths[i]
    return 4 # whole

def fnote(name, t):
    return mu.note.Note(name=name, quarterLength=flength(t))

def match(notes, fingers):
    match = 0
    for i in range(len(notes)):
        c = notes.iloc[i]
        if fingers.iat[i] == c.finger:
            match += 1
    return match

def drawNotes(stream, notes, fingers):
    prev = notes.iloc[0]
    time = 0.0
    for i in range(len(notes)):
        c = notes.iloc[i]
        if(time < c.t0 and c.t0 - time > 1/32):
            r = mu.note.Rest(quarterLength=flength(c.t0 - time))
            stream.append(r)
        n = fnote(c.note, c.t1 - c.t0)
        n.articulations.append(mu.articulations.Fingering(fingers.iat[i]))
        if c.finger != fingers.iat[i]:
            n.style.color = "red"
            n.addLyric(c._finger)
        stream.append(n)
        prev = c
        time = c.t1

def drawMatches(stream, notes, fingerZipped):
    time = 0.0
    for i, fingers in enumerate(fingerZipped):
        c = notes.iloc[i]
        if(time < c.t0 and c.t0 - time > 1/32):
            r = mu.note.Rest(quarterLength=flength(c.t0 - time))
            stream.append(r)
        n = fnote(c.note, c.t1 - c.t0)
        differ = (len(set(fingers)) - 1) / (len(fingers) - 1)
        color = int((1 - pow(1 - differ, 2)) * 255)
        red = "{:02x}".format(color)
        n.style.color = f"#{red}0000"
        stream.append(n)
        time = c.t1
        
def read_PIG(path):
    names = ["id", "t0", "t1", "note", "_n", "_v", "ch", "_finger"]
    table = pd.read_table(path, names = names, skiprows = 1)
    table["finger"] = table["_finger"].apply(lambda s: abs(int(str(s).split("_")[0])))
    return table

def read_fingering(path):
    with open(path) as f:
        st = f.read()
    return pd.Series([int(s) for s in re.split("", st) if s != "" and s != "_" and s != "\n"])

In [2]:
table = read_PIG("./tmp/PianoFingeringDataset/FingeringFiles/001-1_fingering.txt")
table2 = read_PIG("./tmp/PianoFingeringDataset/FingeringFiles/001-2_fingering.txt")
table5 = read_PIG("./tmp/PianoFingeringDataset/FingeringFiles/001-5_fingering.txt")
table8 = read_PIG("./tmp/PianoFingeringDataset/FingeringFiles/001-8_fingering.txt")

czernyFingeringr = read_fingering('./data/bach_inventionen_01_Czerny_fingering_R.txt')
czernyFingeringl = read_fingering('./data/bach_inventionen_01_Czerny_fingering_L.txt')

In [ ]:
rights = table[table.ch == 0]
pr = mu.stream.Part()
drawNotes(pr, rights, czernyFingeringr)

lefts = table[table.ch == 1]
pl = mu.stream.Part()
drawNotes(pl, lefts, czernyFingeringl)

score = mu.stream.Score()
score.insert(0, pr)
score.insert(0, pl)

#score.write('musicxml', './mxl/001-czerny_fingering_vs_1_rl.xml')
m = match(rights, czernyFingeringr) + match(lefts, czernyFingeringl)
print(f"{m} / {len(rights) + len(lefts)}")

In [ ]:
#fingersr = zip(table[table.ch == 0].finger, table2[table.ch == 0].finger, table5[table.ch == 0].finger, table8[table.ch == 0].finger)
#fingersl = zip(table[table.ch == 1].finger, table2[table.ch == 1].finger, table5[table.ch == 1].finger, table8[table.ch == 1].finger)

rights = table[table.ch == 0]
pr = mu.stream.Part()
drawMatches(pr, rights, zip(table[table.ch == 0].finger, czernyFingeringr))

score = mu.stream.Score()
score.insert(0, pr)

score.write('musicxml', './mxl/PIG_differs2.xml')
#m = match(rights, czernyFingeringr) + match(lefts, czernyFingeringl)
#print(f"{m} / {len(rights) + len(lefts)}")

In [46]:
pig_fingers = [table.finger, table2.finger, table5.finger, table8.finger]
for i in range(4):
    str = f"| {i+1} |"
    for j in range(4):
        fingersr = zip(pig_fingers[i], pig_fingers[j])
        cnt = 0
        for _, fs in enumerate(fingersr):
            if len(set(fs)) < len(fs):
                cnt += 1
        rate = "{:.3f}".format(cnt / 469)
        str += f" {rate} |"
    print(str)

| 1 | 1.000 | 0.748 | 0.682 | 0.719 |
| 2 | 0.748 | 1.000 | 0.768 | 0.699 |
| 3 | 0.682 | 0.768 | 1.000 | 0.699 |
| 4 | 0.719 | 0.699 | 0.699 | 1.000 |
